In [3]:
import pandas as pd
features = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/Working_DataFrame.csv')
target = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/CleanedFinal_Target_Data_Combined_resid_Trend')

In [6]:
import numpy as np

# Define all features based on the columns of the features DataFrame
all_features = features.columns.tolist()

# Create a list to store the new features and their correlations
new_feature_correlations = []

# Create a set to keep track of added features
added_features = set()

# Dictionary to store new features
new_features_dict = {}

# Iterate over each pair of features to create new features
for i in range(len(all_features)):
    for j in range(i, len(all_features)):
        feature1 = all_features[i]
        feature2 = all_features[j]
        
        # Skip the 'Datum' column
        if feature1 == 'Datum' or feature2 == 'Datum':
            continue
        
        # Create new features by multiplying and calculating growth
        new_feature_name = f'{feature1}_x_{feature2}'
        new_feature_growth_name = f'{feature1}_growth'
        
        # Multiply features
        new_features_dict[new_feature_name] = features[feature1] * features[feature2]
        
        # Calculate growth (difference to last period)
        new_features_dict[new_feature_growth_name] = features[feature1].diff()

        # Calculate growth (difference to last period) of the multiplied features
        new_feature_mult_growth_name = f'{new_feature_name}_growth'
        new_features_dict[new_feature_mult_growth_name] = new_features_dict[new_feature_name].diff()
        
        # Calculate correlations with the target variable
        correlation_mult = new_features_dict[new_feature_name].corr(target['PM10_Combined_Trend_Residual'])
        correlation_growth = new_features_dict[new_feature_growth_name].corr(target['PM10_Combined_Trend_Residual'])
         
        # Store the correlations if they are high and not already added
        if abs(correlation_mult) > 0.1 and new_feature_name not in added_features:
            new_feature_correlations.append((new_feature_name, correlation_mult))
            added_features.add(new_feature_name)
        if abs(correlation_growth) > 0.1 and new_feature_growth_name not in added_features:
            new_feature_correlations.append((new_feature_growth_name, correlation_growth))
            added_features.add(new_feature_growth_name)

# Concatenate new features to the original DataFrame
features = pd.concat([features, pd.DataFrame(new_features_dict)], axis=1)

# Print the new features with high correlations
for feature, corr in new_feature_correlations:
    print(f'Feature: {feature}, Correlation: {corr}')


Feature: Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevation corrected], Correlation: 0.295880312664073
Feature: Basel Temperature [2 m elevation corrected]_x_Basel Wind Speed [10 m], Correlation: 0.1649281135229291
Feature: Basel Temperature [2 m elevation corrected]_x_660n_sumPW, Correlation: 0.11947271525695315
Feature: Basel Temperature [2 m elevation corrected]_x_Gasverbrauch, Correlation: -0.33163000210476157
Feature: Basel Temperature [2 m elevation corrected]_x_Traffic, Correlation: 0.10683501722873956
Feature: Basel Precipitation Total_x_Hour, Correlation: -0.133496831045994
Feature: Basel Wind Speed [10 m]_x_Basel Wind Speed [10 m], Correlation: -0.13793937770227843
Feature: Basel Wind Speed [10 m]_x_Basel Wind Direction [10 m], Correlation: -0.10142799072307421
Feature: Basel Wind Speed [10 m]_x_Gasverbrauch, Correlation: -0.16278068991896746
Feature: Basel Wind Speed [10 m]_x_Hour, Correlation: -0.16034332300631474
Feature: Basel Wind Direction [1

In [7]:
MostImp = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/5 - Models/4 - LinReg/0 - ModelData/50MostImpFeatures_DF.csv')

In [11]:
# Select the most important features from the features dataframe
important_features = features[top_50_features]

# Concatenate the important features to the MostImp dataframe
MostImp = pd.concat([MostImp, important_features], axis=1)

print(MostImp.columns)

Index(['Datum', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]', 'Stromverbrauch', '350n_sumLief',
       '350v_sumLW', '352v_sumPW', '352v_sumLief',
       ...
       '402v_sumPW', '402v_sumLief', '402v_sumLW', '402n_sumPW',
       '402n_sumLief', '402n_sumLW', '403n_sumPW', '403n_sumLief',
       '403n_sumLW', '403v_sumPW'],
      dtype='object', length=151)


In [12]:
MostImp.to_csv('50MostImp+CombinedFeatures.csv', index=False)